# Graduate Coffee Roulette Algorithm

## Business Problem
- During COVID-19 Lockdowns, people were feeling lack of connection
- People found it hard meeting new people when working from home

## Proposed Solution
- Coffee Roulette, a progam where Graduates meet a new team member every week for four weeks
- Random assignment of mentors to Graduates is easier than cold emailing/messaging
- Asking Graudates which topics and service lines they are interested in to make effective matches

## Technical Solution
A technical solution was required to get the most out of the program without taking an unreasonable amount of someones time to coordinate a roster. The technical solution invovled just two high level steps:
- Use Microsoft Forms to collect information on participants interests, service lines and levels
- Use Python to take the data and automate the rostering based desired metrics:
 - Overlap of interests
 - Diversity of levels
 - Ensuring all mentors involved

### Python Program Overview
- Read in the structured data from Microsoft Form's Excel output
- Created Bi-Partite Network using NetworkX
- Weighted the edges according to optimise desired metrics
- Find the optimal matching to minimise the edge weights
- Repeat
- Output the results

## Outcome
- 45 Graduates met with 4 mentors each
- Both annecdotal evidence and survey results showed the program was appreciated
- Graduate Committee was asked to repeat the activity again the following year

In [ ]:
import networkx as nx
import csv
import matplotlib.pyplot as plt
import math
import numpy as np

In [ ]:
class Coffee_Roulette():    
    def __init__(self, filename):
        # stored properties
        self.filename = None
        self.header = ['ID', 'Name', 'Group', 'Service_Line', 'Level', 'Interests', 'Service_Line_Preference']
        self.data = None
        self.people_dict = dict()
        self.G = None
        self.matches = list()
        self.graduates = list()
        self.mentors = list()
        self.num_matches = 0
        self.output_headers = ['Week', 'Graduate', 'Mentor', 'Service Line', 'Level', 'Shared interests', 'Preferred service line']

        # default edge weighting parameters
        self.interest_weight = 1.0
        self.interest_met_weight = 0.5
        self.service_line_weight = 1.0
        self.service_line_met_weight = 0.5
        self.level_matched_penalty = 0.6
        self.lonely_mentor_weight = 2.0

        self.load_file(filename)
        self.initiate_people_dict()
        self.initiate_graph()

    
    def load_file(self, filename):
        self.filename = filename

        # read data in as list of lists
        with open(self.filename, 'r', encoding='utf8') as fh:
            reader = csv.reader(fh, delimiter = ',')
            file_header = next(reader)

            # account for the strange reading of "ID" in the csv file
            if self.header != ["ID" if "ID" in word else word for word in file_header]:
                raise Exception('Unexpected header row. Expecting:\n"'+', '.join(self.header)+'".')
            
            self.data = [row for row in reader]

            
    def initiate_people_dict(self):
        people = [
            {self.header[i]: row[i] for i, _ in enumerate(self.header)} for row in self.data
        ]

        self.people_dict = {d['ID']: d for d in people}

        for person in people:
            for key in ['Interests','Service_Line_Preference']:
                person[key] = set(person[key].split(';'))
            self.people_dict[person['ID']]['Interests_met'] = set()
            self.people_dict[person['ID']]['Service_Line_Preference_met'] = set()
            self.people_dict[person['ID']]['Levels_met'] = set()
            self.people_dict[person['ID']]['Matches'] = list()
            
            if person['Group'] == 'Graduate':
                self.graduates.append(person['ID'])
            else:
                self.mentors.append(person['ID'])
    

    def initiate_graph(self):
        self.G = nx.Graph()

        for person, person_dict in self.people_dict.items():
            self.G.add_node(person, group = person_dict['Group'])
            
            
    def update_edge_weights(self):
        self.G.remove_edges_from(self.G.edges)
        for g_id in self.graduates:
            for m_id in self.mentors:

                # skip if already a match
                if m_id in self.people_dict[g_id]['Matches'] :
                    continue

                # add an edge between all mentors and graduates
                self.G.add_edge(g_id, m_id, weight = -1)
                
                # add weight to all edges where there is shared interest (less if already matched it)
                self.G[g_id][m_id]['weight'] -= self.interest_weight * len(self.people_dict[g_id]['Interests'].intersection(self.people_dict[m_id]['Interests']))
                self.G[g_id][m_id]['weight'] -= self.interest_met_weight * len(self.people_dict[g_id]['Interests_met'].intersection(self.people_dict[m_id]['Interests']))

                # add weight to edges where mentor is in preferred service line (less if already matched it)
                self.G[g_id][m_id]['weight'] -= self.service_line_weight * (1 * self.people_dict[m_id]['Service_Line'] in self.people_dict[g_id]['Service_Line_Preference'])
                self.G[g_id][m_id]['weight'] -= self.service_line_met_weight * (1 * self.people_dict[m_id]['Service_Line'] in self.people_dict[g_id]['Service_Line_Preference_met'])

                # remove weight from edges where the level has been matched
                self.G[g_id][m_id]['weight'] += self.level_matched_penalty if self.people_dict[m_id]['Level'] in self.people_dict[g_id]['Levels_met'] else 0

                # add weight to edges where a mentor has less than average match
                match_comparison = math.floor(len(self.graduates)/len(self.mentors) * self.num_matches)
                self.G[g_id][m_id]['weight'] -= self.lonely_mentor_weight * (1 * (len([match for match in self.people_dict[m_id]['Matches'] if match != '']) <= match_comparison))
             
            
    def find_match(self):
        # find the match the minimises weights matching every node possible but maximum once per node
        matching = nx.algorithms.bipartite.minimum_weight_full_matching(self.G, top_nodes = None, weight = 'weight')
        
        self.num_matches += 1
        
        tracking_matches = set()
        
        for key, value in matching.items():
            g_id, m_id = sorted((key, value), key = lambda x: self.people_dict[x]['Group'] != 'Graduate')

            coffee_date = (g_id, m_id)
            
            if coffee_date in tracking_matches:
                continue
            
            # else
            tracking_matches.add(coffee_date)
            
            # find shared interests and service line preferences
            shared_interests = '; '.join(list(self.people_dict[g_id]['Interests'].intersection(self.people_dict[m_id]['Interests'])))
            preferred_service_line = '; '.join(list(self.people_dict[g_id]['Service_Line_Preference'].intersection({self.people_dict[m_id]['Service_Line']})))

    
            self.matches.append([
                'Week '+str(self.num_matches),
                g_id+': '+self.people_dict[g_id]['Name'],
                m_id+': '+self.people_dict[m_id]['Name'],
                self.people_dict[m_id]['Service_Line'],
                self.people_dict[m_id]['Level'],
                shared_interests,
                preferred_service_line
            ])

            # update people_dict
            self.people_dict[g_id]['Matches'].append(m_id)
            self.people_dict[m_id]['Matches'].append(g_id)

            self.people_dict[g_id]['Interests_met'].update(self.people_dict[g_id]['Interests'].intersection(self.people_dict[m_id]['Interests']))
            self.people_dict[g_id]['Service_Line_Preference_met'].update(self.people_dict[g_id]['Service_Line_Preference'].intersection({self.people_dict[m_id]['Service_Line']}))
            self.people_dict[g_id]['Levels_met'].add(self.people_dict[m_id]['Level'])
        
        for mentor in self.mentors:
            if len(self.people_dict[mentor]['Matches']) < self.num_matches:
                self.people_dict[mentor]['Matches'].append('')
            
            
    def evaluate(self):
        count_attr = lambda grad, key: len(self.people_dict[grad][key])
        
        # num interests
        interests_met = [count_attr(grad, 'Interests_met') for grad in self.graduates]
        interests = [count_attr(grad, 'Interests') for grad in self.graduates]
        
        # num service line met
        service_lines_met = [count_attr(grad, 'Service_Line_Preference_met') for grad in self.graduates]
        service_lines = [count_attr(grad, 'Service_Line_Preference') for grad in self.graduates]
        
        # num levels
        levels = [count_attr(grad, 'Levels_met') for grad in self.graduates]
        
        # num matches for mentors
        mentor_matches = [len(list(filter(lambda x: x != '', self.people_dict[mentor]['Matches']))) for mentor in self.mentors]
        
        overall_interests_met = [
            num_intrs + num_sls for num_intrs, num_sls in zip(interests_met, service_lines_met)
        ]
        
        overall_interests = [
            num_intrs + num_sls for num_intrs, num_sls in zip(interests, service_lines)
        ]
        
        overall_interests_pct = [
            num_met / num_total for num_met, num_total in zip(overall_interests_met, overall_interests)
        ]
        
        # set up grid of graphs
        fig, ((left_top, mid_top, right_top), (left_bot, mid_bot, right_bot)) = plt.subplots(2,3, figsize = (16, 6)) 
        
        # interests met graph
        left_top.set_title('% Interests met')
        left_top.hist(overall_interests_pct)
        left_bot.boxplot(overall_interests_pct)
        
        # different levels met
        mid_top.set_title('# levels')
        mid_top.hist(levels)
        mid_bot.boxplot(levels)
        
        # number of matches each mentor has
        right_top.set_title('# Mentor matches')
        right_top.hist(mentor_matches)
        right_bot.boxplot(mentor_matches)
        
        plt.draw()
        
        # print min and average statistics     
        print('Field', 'min','average', sep=' | ')
        print('_________________________________')
        print('Overall interests', np.min(overall_interests_pct), np.average(overall_interests_pct), sep=' | ')
        print('levels', np.min(levels), np.average(levels), sep=' | ')
        print('mentor matches', np.min(mentor_matches), np.average(mentor_matches), sep=' | ')
        
        
    def display_week_matches(self, week):
        subset = [(row[1].split(':')[0],row[2].split(':')[0]) for row in self.matches if row[0].lower() == week.lower()]

        # output matches
        grad_set = nx.bipartite.sets(self.G)[0]
        pos = nx.bipartite_layout(self.G, grad_set)
        node_colours = ['k' if node[1] == 'Graduate' else 'b' for node in self.G.nodes(data = 'group')]
        nx.draw_networkx(self.G, node_color = node_colours, pos=pos, edgelist=subset)
    
    
    def output_match_list(self, filename):
        with open(filename,'w') as fh:
            writer = csv.writer(fh, delimiter = ',')
            writer.writerow(self.output_headers)
            writer.writerows(self.matches)
            
            
    def output_registration_list(self, filename):
        with open(filename, 'w') as fh:
            writer = csv.writer(fh, delimiter = ',')
            registration_list_header = self.header.copy()
            for i in range(self.num_matches):
                registration_list_header.append('Week {}'.format(i+1))
            writer.writerow(registration_list_header)
            for row in self.data:
                row_id = row[0]
                match_ids = self.people_dict[row_id]['Matches']
                match_names = ['' if m_id == '' else m_id + ': '+self.people_dict[m_id]['Name'] for m_id in match_ids]
                writer.writerow(row+match_names)

                
    def output_interests_and_service_line_data(self):
        header_interest = ['ID', 'Group', 'Service Line', 'Level', 'Interest']
        interests = []
        header_service_line_preferences = ['ID', 'Group', 'Service Line', 'Level', 'Service Line Preference']
        service_line_preferences = []
        for person in self.people_dict.values():
            
            for interest in person['Interests']:
                interests.append([
                    person['ID'],
                    person['Group'],
                    person['Service_Line'],
                    person['Level'],
                    interest
                ])
            
            for sl_preference in person['Service_Line_Preference']:
                service_line_preferences.append([
                    person['ID'],
                    person['Group'],
                    person['Service_Line'],
                    person['Level'],
                    sl_preference
                ])
                
        with open('interests.csv', 'w') as fh:
            writer = csv.writer(fh, delimiter = ',')
            writer.writerow(header_interest)
            writer.writerows(interests)
            
        with open('service_line_preferences.csv', 'w') as fh:
            writer = csv.writer(fh, delimiter = ',')
            writer.writerow(header_service_line_preferences)
            writer.writerows(service_line_preferences)
            

In [ ]:
Coff = Coffee_Roulette('registrations.csv')

# defaults
'''
Coff.interest_weight = 1.0
Coff.interest_met_weight = 0.5
Coff.service_line_weight = 1.0
Coff.service_line_met_weight = 0.5
Coff.level_matched_penalty = 0.6
Coff.lonely_mentor_weight = 2.0
'''

for i in range(4):
    Coff.update_edge_weights()
    Coff.find_match()
Coff.evaluate()

In [ ]:
Coff = Coffee_Roulette('registrations.csv')
    
Coff.interest_weight = 0
Coff.interest_met_weight = 0
Coff.service_line_weight = 0
Coff.service_line_met_weight = 0
Coff.level_matched_penalty = 0
Coff.lonely_mentor_weight = 0
    
for i in range(4):
    Coff.update_edge_weights()
    Coff.find_match()
Coff.evaluate()

In [ ]:
Coff = Coffee_Roulette('registrations.csv')
    
Coff.interest_weight = 1
Coff.interest_met_weight = 0
Coff.service_line_weight = 1
Coff.service_line_met_weight = 0
Coff.level_matched_penalty = 0
Coff.lonely_mentor_weight = 0
    
for i in range(4):
    Coff.update_edge_weights()
    Coff.find_match()
Coff.evaluate()

In [ ]:
Coff = Coffee_Roulette('registrations.csv')
    
Coff.interest_weight = 1
Coff.interest_met_weight = 0
Coff.service_line_weight = 1
Coff.service_line_met_weight = 0
Coff.level_matched_penalty = 1
Coff.lonely_mentor_weight = 0
    
for i in range(4):
    Coff.update_edge_weights()
    Coff.find_match()
Coff.evaluate()

In [ ]:
Coff = Coffee_Roulette('registrations.csv')
    
Coff.interest_weight = 1
Coff.interest_met_weight = 0
Coff.service_line_weight = 1
Coff.service_line_met_weight = 0
Coff.level_matched_penalty = 1
Coff.lonely_mentor_weight = 1.5
    
for i in range(4):
    Coff.update_edge_weights()
    Coff.find_match()
Coff.evaluate()

In [ ]:
Coff.output_match_list('output.csv')

In [ ]:
Coff.output_registration_list('registration_list_output.csv')

In [ ]:
Coff.output_interests_and_service_line_data()